In [0]:
df = spark.read.format("csv")\
  .option("header", "true")\
  .option("inferSchema", "true")\
  .load("/FileStore/tables/2010_12_01-1.csv")
# /FileStore/tables/2010_12_01-1.csv
df.printSchema()
df.createOrReplaceTempView("dfTable")


root
-- InvoiceNo: string (nullable = true)
-- StockCode: string (nullable = true)
-- Description: string (nullable = true)
-- Quantity: integer (nullable = true)
-- InvoiceDate: string (nullable = true)
-- UnitPrice: double (nullable = true)
-- CustomerID: double (nullable = true)
-- Country: string (nullable = true)

In [0]:
df.head(5)

Out[3]: [Row(InvoiceNo='536365', StockCode='85123A', Description='WHITE HANGING HEART T-LIGHT HOLDER', Quantity=6, InvoiceDate='2010-12-01 08:26:00', UnitPrice=2.55, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='71053', Description='WHITE METAL LANTERN', Quantity=6, InvoiceDate='2010-12-01 08:26:00', UnitPrice=3.39, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84406B', Description='CREAM CUPID HEARTS COAT HANGER', Quantity=8, InvoiceDate='2010-12-01 08:26:00', UnitPrice=2.75, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029G', Description='KNITTED UNION FLAG HOT WATER BOTTLE', Quantity=6, InvoiceDate='2010-12-01 08:26:00', UnitPrice=3.39, CustomerID=17850.0, Country='United Kingdom'),
 Row(InvoiceNo='536365', StockCode='84029E', Description='RED WOOLLY HOTTIE WHITE HEART.', Quantity=6, InvoiceDate='2010-12-01 08:26:00', UnitPrice=3.39, CustomerID=17850.0, Country='United Kingdom')]

lit function -converts a type in another language to its corresponding Spark representation

In [0]:
from pyspark.sql.functions import lit
df.select(lit(5), lit("five"), lit(5.0))


Out[2]: DataFrame[5: int, five: string, 5.0: double]

specify equality:

In [0]:
from pyspark.sql.functions import col
df.where(col("InvoiceNo") != 536365)\
  .select("InvoiceNo", "Description")\
  .show(5, False)


+---------+-----------------------------+
InvoiceNo|Description |
+---------+-----------------------------+
536366 |HAND WARMER UNION JACK |
536366 |HAND WARMER RED POLKA DOT |
536367 |ASSORTED COLOUR BIRD ORNAMENT|
536367 |POPPY'S PLAYHOUSE BEDROOM |
536367 |POPPY'S PLAYHOUSE KITCHEN |
+---------+-----------------------------+
only showing top 5 rows

Chain together filters as a sequential filter:

In [0]:
from pyspark.sql.functions import instr
priceFilter = col("UnitPrice") > 600
descripFilter = instr(df.Description, "POSTAGE") >= 1
df.where(df.StockCode.isin("DOT")).where(priceFilter | descripFilter).show()


+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
InvoiceNo|StockCode| Description|Quantity| InvoiceDate|UnitPrice|CustomerID| Country|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+
 536544| DOT|DOTCOM POSTAGE| 1|2010-12-01 14:32:00| 569.77| null|United Kingdom|
 536592| DOT|DOTCOM POSTAGE| 1|2010-12-01 17:06:00| 607.49| null|United Kingdom|
+---------+---------+--------------+--------+-------------------+---------+----------+--------------+

Filter a DataFrame by specifying a Boolean column:

In [0]:
from pyspark.sql.functions import instr
DOTCodeFilter = col("StockCode") == "DOT"
priceFilter = col("UnitPrice") > 600
descripFilter = instr(col("Description"), "POSTAGE") >= 1
df.withColumn("isExpensive", DOTCodeFilter & (priceFilter | descripFilter))\
  .where("isExpensive")\
  .select("unitPrice", "isExpensive").show(5)


+---------+-----------+
unitPrice|isExpensive|
+---------+-----------+
 569.77| true|
 607.49| true|
+---------+-----------+

Expressed as a where clause a la SQL. Easier to express filters as SQL statements than using the programmatic DataFrame interface and Spark SQL allows us to do this without paying any performance penalty:

In [0]:
from pyspark.sql.functions import expr
df.withColumn("isExpensive", expr("NOT UnitPrice <= 250"))\
  .where("isExpensive")\
  .select("Description", "UnitPrice").show(5)


+--------------+---------+
 Description|UnitPrice|
+--------------+---------+
DOTCOM POSTAGE| 569.77|
DOTCOM POSTAGE| 607.49|
+--------------+---------+

If you’re working with null data when creating Boolean expressions, must perform a null-safe equivalence test:

In [0]:
df.where(col("Description").eqNullSafe("hello")).show()

fabricate a contrived example 
* let’s imagine that we found out that we mis-recorded the quantity in our retail dataset 
* true quantity is equal to (the current quantity * the unitprice) + 5.

In [0]:
from pyspark.sql.functions import expr, pow
fabricatedQuantity = pow(col("Quantity") * col("UnitPrice"), 2) + 5
df.select(expr("CustomerId"), fabricatedQuantity.alias("realQuantity")).show(2)


+----------+------------------+
CustomerId| realQuantity|
+----------+------------------+
 17850.0|239.08999999999997|
 17850.0| 418.7156|
+----------+------------------+
only showing top 2 rows

as a SQLexpression:

In [0]:
df.selectExpr(
  "CustomerId",
  "(POWER((Quantity * UnitPrice), 2.0) + 5) as realQuantity").show(2)


+----------+------------------+
CustomerId| realQuantity|
+----------+------------------+
 17850.0|239.08999999999997|
 17850.0| 418.7156|
+----------+------------------+
only showing top 2 rows

rounding -  easiest to cast the value to an integer. Spark has more detailed functions for performing this explicitly and to a certain level of precision. In the following example, we round to one decimal place. By default, the round function rounds up if you’re exactly in between two numbers. You can round down by using the **bround**

In [0]:
# in Python
from pyspark.sql.functions import lit,round,bround
df.select(round(lit("2.5")),bround(lit("2.5"))).show(2)

+-------------+--------------+
round(2.5, 0)|bround(2.5, 0)|
+-------------+--------------+
 3.0| 2.0|
 3.0| 2.0|
+-------------+--------------+
only showing top 2 rows

Compute the correlation of two columns.See the Pearson correlation coefficient for two columns (to see if cheaper things are typically bought in greater quantities). Can do this through a function as well as through the DataFrame statistic methods:

In [0]:
from pyspark.sql.functions import corr
df.stat.corr("Quantity", "UnitPrice")
df.select(corr("Quantity", "UnitPrice")).show()


+-------------------------+
corr(Quantity, UnitPrice)|
+-------------------------+
 -0.04112314436835551|
+-------------------------+

Summary Stats:

In [0]:
df.describe().show()


+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
summary| InvoiceNo| StockCode| Description| Quantity| InvoiceDate| UnitPrice| CustomerID| Country|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+
 count| 3108| 3108| 3098| 3108| 3108| 3108| 1968| 3108|
 mean| 536516.684944841|27834.304044117645| null| 8.627413127413128| null| 4.151946589446603|15661.388719512195| null|
 stddev|72.89447869788873|17407.897548583845| null|26.371821677029203| null|15.638659854603892|1854.4496996893627| null|
 min| 536365| 10002| 4 PURPLE FLOCK D...| -24|2010-12-01 08:26:00| 0.0| 12431.0| Australia|
 max| C536548| POST|ZINC WILLIE WINKI...| 600|2010-12-01 17:35:00| 607.49| 18229.0|United Kingdom|
+-------+-----------------+------------------+--------------------+------------------+-------------------+------------------+------------------+--------------+

individual functions:

In [0]:
from pyspark.sql.functions import count, mean, stddev_pop, min, max


Calculate either exact or approximate quantiles of your data using the approxQuantile method:

In [0]:
colName = "UnitPrice"
quantileProbs = [0.5]
relError = 0.05
df.stat.approxQuantile("UnitPrice", quantileProbs, relError) # 2.51


Out[18]: [2.51]

see a cross-tabulation:

In [0]:
df.stat.crosstab("StockCode", "Quantity").show()


+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
StockCode_Quantity| -1|-10|-12| -2|-24| -3| -4| -5| -6| -7| 1| 10|100| 11| 12|120|128| 13| 14|144| 15| 16| 17| 18| 19|192| 2| 20|200| 21|216| 22| 23| 24| 25|252| 27| 28|288| 3| 30| 32| 33| 34| 36|384| 4| 40|432| 47| 48|480| 5| 50| 56| 6| 60|600| 64| 7| 70| 72| 8| 80| 9| 96|
+------------------+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+---+
 22578| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 21327| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 2| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 22064| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 21080| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1|
 22219| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 3| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 21908| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 22818| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 15056BL| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 72817| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 22545| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 22988| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0|
 22274| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 20750| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 2| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 82616C| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 21703| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 22899| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 

frequent item pairs:

In [0]:
df.stat.freqItems(["StockCode", "Quantity"]).show()


+--------------------+--------------------+
 StockCode_freqItems| Quantity_freqItems|
+--------------------+--------------------+
[90214E, 20728, 2...|[200, 128, 23, 32...|
+--------------------+--------------------+

Add a unique ID to each row by using the function monotonically_increasing_id. This function generates a unique value for each row, starting with 0:

In [0]:
from pyspark.sql.functions import monotonically_increasing_id
df.select(monotonically_increasing_id()).show(2)


+-----------------------------+
monotonically_increasing_id()|
+-----------------------------+
 0|
 1|
+-----------------------------+
only showing top 2 rows

initcap - function will capitalize every word in a given string when that word is separated from another by a space:

In [0]:
from pyspark.sql.functions import initcap
df.select(initcap(col("Description"))).show()


+--------------------+
initcap(Description)|
+--------------------+
White Hanging Hea...|
 White Metal Lantern|
Cream Cupid Heart...|
Knitted Union Fla...|
Red Woolly Hottie...|
Set 7 Babushka Ne...|
Glass Star Froste...|
Hand Warmer Union...|
Hand Warmer Red P...|
Assorted Colour B...|
Poppy's Playhouse...|
Poppy's Playhouse...|
Feltcraft Princes...|
Ivory Knitted Mug...|
Box Of 6 Assorted...|
Box Of Vintage Ji...|
Box Of Vintage Al...|
Home Building Blo...|
Love Building Blo...|
Recipe Box With M...|
+--------------------+
only showing top 20 rows

Casting Cases:

In [0]:
from pyspark.sql.functions import lower, upper
df.select(col("Description"),
    lower(col("Description")),
    upper(lower(col("Description")))).show(2)


+--------------------+--------------------+-------------------------+
 Description| lower(Description)|upper(lower(Description))|
+--------------------+--------------------+-------------------------+
WHITE HANGING HEA...|white hanging hea...| WHITE HANGING HEA...|
 WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN|
+--------------------+--------------------+-------------------------+
only showing top 2 rows

Adding or removing spaces around a string - you can do this by using lpad, ltrim, rpad and rtrim, trim:

In [0]:
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim
df.select(
    ltrim(lit("    HELLO    ")).alias("ltrim"),
    rtrim(lit("    HELLO    ")).alias("rtrim"),
    trim(lit("    HELLO    ")).alias("trim"),
    lpad(lit("HELLO"), 3, " ").alias("lp"),
    rpad(lit("HELLO"), 10, " ").alias("rp")).show(2)


+---------+---------+-----+---+----------+
 ltrim| rtrim| trim| lp| rp|
+---------+---------+-----+---+----------+
HELLO | HELLO|HELLO|HEL|HELLO |
HELLO | HELLO|HELLO|HEL|HELLO |
+---------+---------+-----+---+----------+
only showing top 2 rows

RegEx - replace substitute color names in our description column

In [0]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
  regexp_replace(col("Description"), regex_string, "COLOR").alias("color_clean"),
  col("Description")).show(7)


+--------------------+--------------------+
 color_clean| Description|
+--------------------+--------------------+
COLOR HANGING HEA...|WHITE HANGING HEA...|
 COLOR METAL LANTERN| WHITE METAL LANTERN|
CREAM CUPID HEART...|CREAM CUPID HEART...|
KNITTED UNION FLA...|KNITTED UNION FLA...|
COLOR WOOLLY HOTT...|RED WOOLLY HOTTIE...|
SET 7 BABUSHKA NE...|SET 7 BABUSHKA NE...|
GLASS STAR FROSTE...|GLASS STAR FROSTE...|
+--------------------+--------------------+
only showing top 7 rows

Spark provides the translate function to replace these values. This is done at the character level and will replace all instances of a character with the indexed character in the replacement string (leetspeak example, lol):

In [0]:
from pyspark.sql.functions import translate
df.select(translate(col("Description"), "LEET", "1337"),col("Description"))\
  .show(2)


+----------------------------------+--------------------+
translate(Description, LEET, 1337)| Description|
+----------------------------------+--------------------+
 WHI73 HANGING H3A...|WHITE HANGING HEA...|
 WHI73 M37A1 1AN73RN| WHITE METAL LANTERN|
+----------------------------------+--------------------+
only showing top 2 rows

extracting first mentioned color:

In [0]:
from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
     regexp_extract(col("Description"), extract_str, 1).alias("color_clean"),
     col("Description")).show(2)


+-----------+--------------------+
color_clean| Description|
+-----------+--------------------+
 WHITE|WHITE HANGING HEA...|
 WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows

Check for  existence. We can do this with the contains method on each column. This will return a Boolean declaring whether the value you specify is in the column’s string:

In [0]:
from pyspark.sql.functions import instr
containsBlack = instr(col("Description"), "BLACK") >= 1
containsWhite = instr(col("Description"), "WHITE") >= 1
df.withColumn("hasSimpleColor", containsBlack | containsWhite)\
  .where("hasSimpleColor")\
  .select("Description").show(3, False)


+----------------------------------+
Description |
+----------------------------------+
WHITE HANGING HEART T-LIGHT HOLDER|
WHITE METAL LANTERN |
RED WOOLLY HOTTIE WHITE HEART. |
+----------------------------------+
only showing top 3 rows

locate - returns the integer location (1 based location),  then converts that to a Boolean before using it as the same basic feature:

In [0]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
  return locate(color_string.upper(), column)\
          .cast("boolean")\
          .alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
  .select("Description").show(3, False)


+----------------------------------+
Description |
+----------------------------------+
WHITE HANGING HEART T-LIGHT HOLDER|
WHITE METAL LANTERN |
RED WOOLLY HOTTIE WHITE HEART. |
+----------------------------------+
only showing top 3 rows

get the current date and the current timestamps:

In [0]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(10)\
  .withColumn("today", current_date())\
  .withColumn("now", current_timestamp())
dateDF.createOrReplaceTempView("dateTable")


Add and subtract five days from today. These functions take a column and then the number of days to either add or subtract as the arguments:

In [0]:
from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub(col("today"), 5), date_add(col("today"), 5)).show(1)


+------------------+------------------+
date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
 2020-11-01| 2020-11-11|
+------------------+------------------+
only showing top 1 row

* datediff function - will return the number of days in between two dates
* months_between - gives you the number of months between two dates

In [0]:
from pyspark.sql.functions import datediff, months_between, to_date
dateDF.withColumn("week_ago", date_sub(col("today"), 7))\
  .select(datediff(col("week_ago"), col("today"))).show(1)

dateDF.select(
    to_date(lit("2016-01-01")).alias("start"),
    to_date(lit("2017-05-22")).alias("end"))\
  .select(months_between(col("start"), col("end"))).show(1)


+-------------------------+
datediff(week_ago, today)|
+-------------------------+
 -7|
+-------------------------+
only showing top 1 row

+--------------------------------+
months_between(start, end, true)|
+--------------------------------+
 -16.67741935|
+--------------------------------+
only showing top 1 row

In [0]:
from pyspark.sql.functions import to_date, lit
spark.range(5).withColumn("date", lit("2017-01-01"))\
  .select(to_date(col("date"))).show(1)


+---------------+
to_date(`date`)|
+---------------+
 2017-01-01|
+---------------+
only showing top 1 row

Spark will not throw an error if it cannot parse the date; rather, it will just return null. This can be a bit tricky in larger pipelines because you might be expecting your data in one format and getting it in another. To illustrate, let’s take a look at the date format that has switched from year-month-day to year-day-month. Spark will fail to parse this date and silently return null instead:

In [0]:
dateDF.select(to_date(lit("2016-20-12")),to_date(lit("2017-12-11"))).show(1)

+---------------------+---------------------+
to_date('2016-20-12')|to_date('2017-12-11')|
+---------------------+---------------------+
 null| 2017-12-11|
+---------------------+---------------------+
only showing top 1 row

In [0]:
from pyspark.sql.functions import to_date
dateFormat = "yyyy-dd-MM"
cleanDateDF = spark.range(1).select(
    to_date(lit("2017-12-11"), dateFormat).alias("date"),
    to_date(lit("2017-20-12"), dateFormat).alias("date2"))
cleanDateDF.createOrReplaceTempView("dateTable2")


to_timestamp - which always requires a format to be specified

In [0]:
from pyspark.sql.functions import to_timestamp
cleanDateDF.select(to_timestamp(col("date"), dateFormat)).show()


+----------------------------------+
to_timestamp(`date`, 'yyyy-dd-MM')|
+----------------------------------+
 2017-11-12 00:00:00|
+----------------------------------+

Select the first non-null value from a set of columns by using the coalesce function. In this case, there are no null values, so it simply returns the first column:

In [0]:
from pyspark.sql.functions import coalesce
df.select(coalesce(col("Description"), col("CustomerId"))).show()


+---------------------------------+
coalesce(Description, CustomerId)|
+---------------------------------+
 WHITE HANGING HEA...|
 WHITE METAL LANTERN|
 CREAM CUPID HEART...|
 KNITTED UNION FLA...|
 RED WOOLLY HOTTIE...|
 SET 7 BABUSHKA NE...|
 GLASS STAR FROSTE...|
 HAND WARMER UNION...|
 HAND WARMER RED P...|
 ASSORTED COLOUR B...|
 POPPY'S PLAYHOUSE...|
 POPPY'S PLAYHOUSE...|
 FELTCRAFT PRINCES...|
 IVORY KNITTED MUG...|
 BOX OF 6 ASSORTED...|
 BOX OF VINTAGE JI...|
 BOX OF VINTAGE AL...|
 HOME BUILDING BLO...|
 LOVE BUILDING BLO...|
 RECIPE BOX WITH M...|
+---------------------------------+
only showing top 20 rows

In [0]:
df.na.drop("all", subset=["StockCode", "InvoiceNo"])


fill function - you can fill one or more columns with a set of values. This can be done by specifying a map—that is a particular value and a set of columns

In [0]:
df.na.fill("all", subset=["StockCode", "InvoiceNo"])


Out[42]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [0]:
fill_cols_vals = {"StockCode": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals)


Out[43]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

Replace - replace all values in a certain column according to their current value. The only requirement is that this value be the same type as the original value.

In [0]:
df.na.replace([""], ["UNKNOWN"], "Description")


Out[45]: DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: int, InvoiceDate: string, UnitPrice: double, CustomerID: double, Country: string]

In [0]:
from pyspark.sql.functions import struct
complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))
complexDF.createOrReplaceTempView("complexDF")


Split function and specify the delimiter:

In [0]:
from pyspark.sql.functions import split
df.select(split(col("Description"), " "), col("Description")).show(2)


+-------------------------+--------------------+
split(Description, , -1)| Description|
+-------------------------+--------------------+
 [WHITE, HANGING, ...|WHITE HANGING HEA...|
 [WHITE, METAL, LA...| WHITE METAL LANTERN|
+-------------------------+--------------------+
only showing top 2 rows

Manipulate this complex type as another column:

In [0]:
df.select(split(col("Description"), " ").alias("array_col"))\
  .selectExpr("array_col[0]").show(2)


+------------+
array_col[0]|
+------------+
 WHITE|
 WHITE|
+------------+
only showing top 2 rows

In [0]:
from pyspark.sql.functions import size
df.select(size(split(col("Description"), " "))).show(2) # shows 5 and 3


+-------------------------------+
size(split(Description, , -1))|
+-------------------------------+
 5|
 3|
+-------------------------------+
only showing top 2 rows

In [0]:
from pyspark.sql.functions import array_contains
df.select(array_contains(split(col("Description"), " "), "WHITE")).show(2)


+------------------------------------------------+
array_contains(split(Description, , -1), WHITE)|
+------------------------------------------------+
 true|
 true|
+------------------------------------------------+
only showing top 2 rows

In [0]:
from pyspark.sql.functions import split, explode

df.withColumn("splitted", split(col("Description"), " "))\
  .withColumn("exploded", explode(col("splitted")))\
  .select("Description", "InvoiceNo", "exploded").show(4)


+--------------------+---------+--------+
 Description|InvoiceNo|exploded|
+--------------------+---------+--------+
WHITE HANGING HEA...| 536365| WHITE|
WHITE HANGING HEA...| 536365| HANGING|
WHITE HANGING HEA...| 536365| HEART|
WHITE HANGING HEA...| 536365| T-LIGHT|
+--------------------+---------+--------+
only showing top 4 rows

Maps are created by using the map function and key-value pairs of columns. You then can select them just like you might select from an array:

In [0]:
from pyspark.sql.functions import create_map
df.select(create_map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .show(2)


+--------------------+
 complex_map|
+--------------------+
[WHITE HANGING HE...|
[WHITE METAL LANT...|
+--------------------+
only showing top 2 rows

In [0]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("complex_map['WHITE METAL LANTERN']").show(2)


--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4307127719999333> in <module> 
 ----> 1 df . select ( map ( col ( "Description" ) , col ( "InvoiceNo" ) ) . alias ( "complex_map" ) ) \ 
 2 . selectExpr ( "complex_map['WHITE METAL LANTERN']" ) . show ( 2 ) 

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 351 
 352 def __iter__ ( self ) : 
 --> 353 raise TypeError ( "Column is not iterable" ) 
 354 
 355 # string methods 

 TypeError : Column is not iterable

In [0]:
df.select(map(col("Description"), col("InvoiceNo")).alias("complex_map"))\
  .selectExpr("explode(complex_map)").show(2)


--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-4307127719999334> in <module> 
 ----> 1 df . select ( map ( col ( "Description" ) , col ( "InvoiceNo" ) ) . alias ( "complex_map" ) ) \ 
 2 . selectExpr ( "explode(complex_map)" ) . show ( 2 ) 

 /databricks/spark/python/pyspark/sql/column.py in __iter__ (self) 
 351 
 352 def __iter__ ( self ) : 
 --> 353 raise TypeError ( "Column is not iterable" ) 
 354 
 355 # string methods 

 TypeError : Column is not iterable

In [0]:
jsonDF = spark.range(1).selectExpr("""
  '{"myJSONKey" : {"myJSONValue" : [1, 2, 3]}}' as jsonString""")


In [0]:
from pyspark.sql.functions import get_json_object, json_tuple

jsonDF.select(
    get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column",
    json_tuple(col("jsonString"), "myJSONKey")).show(2)


File "<command-4307127719999336>" , line 4 
 get_json_object(col("jsonString"), "$.myJSONKey.myJSONValue[1]") as "column", 
 ^
 SyntaxError : invalid syntax

In [0]:
from pyspark.sql.functions import to_json
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")))


Out[60]: DataFrame[to_json(myStruct): string]

In [0]:
from pyspark.sql.functions import from_json
from pyspark.sql.types import *
parseSchema = StructType((
  StructField("InvoiceNo",StringType(),True),
  StructField("Description",StringType(),True)))
df.selectExpr("(InvoiceNo, Description) as myStruct")\
  .select(to_json(col("myStruct")).alias("newJSON"))\
  .select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)


+--------------------+--------------------+
 from_json(newJSON)| newJSON|
+--------------------+--------------------+
[536365, WHITE HA...|{"InvoiceNo":"536...|
[536365, WHITE ME...|{"InvoiceNo":"536...|
+--------------------+--------------------+
only showing top 2 rows

In [0]:
udfExampleDF = spark.range(5).toDF("num")
def power3(double_value):
  return double_value ** 3
power3(2.0)


Out[62]: 8.0

In [0]:
from pyspark.sql.functions import udf
power3udf = udf(power3)


In [0]:
from pyspark.sql.functions import col
udfExampleDF.select(power3udf(col("num"))).show(2)


+-----------+
power3(num)|
+-----------+
 0|
 1|
+-----------+
only showing top 2 rows

register a function to make it available as a DataFrame function:

In [0]:
from pyspark.sql.types import IntegerType, DoubleType
spark.udf.register("power3py", power3, DoubleType())


Out[66]: <function __main__.power3(double_value)>

If you specify the type that doesn’t align with the actual type returned by the function, Spark will not throw an error but will just return null to designate a failure:

In [0]:
udfExampleDF.selectExpr("power3py(num)").show(2)
# registered via Python


+-------------+
power3py(num)|
+-------------+
 null|
 null|
+-------------+
only showing top 2 rows